
1. Импортируем все необходимое для работы


In [ ]:
!pip install pymorphy2
import os
import json
import string

from pymystem3 import Mystem
from collections import Counter
from string import punctuation
from collections import Counter
from sklearn.metrics import accuracy_score

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


2. Функция: читаем файл

In [49]:
def import_text(text_f):
    with open(text_f, 'r', encoding="utf-8") as text:
        text_raw = text.read()
    return text_raw.split("*")

3. Функция: токенизируем

In [50]:
def task_mystem(text, list_lex: list):
    tokens = m.analyze(text.lower())
    tokens = [token['analysis'][0]['lex'] for token in tokens if 'analysis' in token and len(token['analysis']) > 0]
    for item in tokens:
        list_lex.append(item)

4. Берем текст из файлов (материал собирался вручную).

In [ ]:
list_good_raw = import_text("good.txt")
list_bad_raw = import_text("bad.txt")
list_test_raw = import_text("test1.txt")

list_good_lex = []
list_bad_lex = []

5. Сравниваем содержимое после разбивки.

In [51]:
list_only_good_lex = list(set([lex for lex in list_good_lex if lex not in list_bad_lex]))
list_only_bad_lex = list(set([lex for lex in list_bad_lex if lex not in list_good_lex]))

list_lex = {'good': list_only_good_lex, 'bad': list_only_bad_lex}

6. Функция: обучаем и проверяем детектор.

In [52]:
def simple_review_detect(freq_lists, text):
    counts = Counter()
    for review, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in m.lemmatize(text.lower()):
            counts[review] += int(freq_list[word] > 0)
    return counts.most_common()

In [ ]:
def test_simple_review_detect(freq_lists, text, review):
    results = []
    gold = []
    predicted_review = simple_review_detect(freq_lists, text)
    results.append(predicted_lang[0][0])
    gold.append(review)
    print("RESULTS:", results)
    print("Accuracy: %.4f" % accuracy_score(results, gold))


**7**. Проверяем работу наших функций. (У меня в коллабе возникает ошибка Broken Pipe, но на локальном уровне все работает)

In [55]:
for text in list_good_raw:
    test_simple_review_detect(list_lex, text, review='good')

BrokenPipeError: ignored

In [ ]:
for text in list_bad_raw:
    test_simple_review_detect(list_lex, text, review='bad')

Как можно улучшить?
1. Семантическая проверка текста перед тем, как выносить решение о хорошем/плохом отзыве.
2. Автоматически подгружать рандомный отзыв на продукт с сайта.